In [23]:
import pandas as pd
from datetime import timedelta, datetime
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable 
from torch.utils.data import DataLoader, Dataset
import numpy as np

In [24]:
# Format into set ready for training and eval
df = pd.read_csv("feat_eng_1800.csv")

# Days in the past
days_in_past = 0

# [{ID, [entries], label}]
data = []
X = dict()
Y = dict()

df["timestamp"] = pd.to_datetime(df["timestamp"])

for id in df["ID"].unique():
    X[id] = dict()
    Y[id] = dict()
    df_id = df[df["ID"] == id]
    for day in df_id["timestamp"].dt.floor("D").unique():
        
        df_id_days = df_id[((df_id["timestamp"].dt.date <= day.date()) & (df_id["timestamp"].dt.date >= (day - timedelta(days=days_in_past)).date()))].sort_values(by="timestamp", ascending=False)
        label = df_id_days["next_mood"].iloc[0]
        df_id_days = df_id_days.sort_values(by="timestamp", ascending=True).drop(columns=["ID"])

        # being an RNN, timestamp shouldn't be needed since the order is the important
        df_id_days["timestamp"] = df_id_days["timestamp"].astype(int) // 10**11
        df_id_days["timestamp"] = df_id_days["timestamp"] - df_id_days["timestamp"].min()

        df_id_days = df_id_days.drop(columns="Unnamed: 0").reset_index(drop=True)

        # Using only one value since it will be the next day predicted value
        X[id][day] , Y[id][day] = df_id_days.drop(columns="next_mood"), df_id_days["next_mood"][0]

        #raw_tuples = list(df_id_days.itertuples(index=False, name=None))
        #print(len(raw_tuples[0]))
        #data.append({"ID": id, "entries": raw_tuples, "label": label})


In [25]:
import random

train_ids = random.sample(list(X.keys()), int(len(X)*0.9))

X_train = []
Y_train = []

X_test = []
Y_test = []

for id in X.keys():
    is_train = True if id in train_ids else False
    for day in X[id].keys():
        if is_train:
            X_train.append(X[id][day])
            Y_train.append(Y[id][day])
        else:
            X_test.append(X[id][day])
            Y_test.append(Y[id][day])

X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(1126, 48, 22)
(1126,)
(142, 48, 22)
(142,)


In [26]:
# Convert to pytorch tensors
X_train_tensors = Variable(torch.Tensor(X_train))
X_test_tensors = Variable(torch.Tensor(X_test))

Y_train_tensors = Variable(torch.Tensor(Y_train))
Y_test_tensors = Variable(torch.Tensor(Y_test))

print(X_train_tensors.shape)
print(X_test_tensors.shape) 

print(Y_train_tensors.shape)
print(Y_test_tensors.shape) 

# Reshaping to rows, timestamps, features
X_train_tensors_final = torch.reshape(X_train_tensors,   
                                      (X_train_tensors.shape[0], 48, 
                                       X_train_tensors.shape[2]))
X_test_tensors_final = torch.reshape(X_test_tensors,  
                                     (X_test_tensors.shape[0], 48, 
                                      X_test_tensors.shape[2])) 

print("Training Shape:", X_train_tensors_final.shape, Y_train_tensors.shape)
print("Testing Shape:", X_test_tensors_final.shape, Y_test_tensors.shape) 

torch.Size([1126, 48, 22])
torch.Size([142, 48, 22])
torch.Size([1126])
torch.Size([142])
Training Shape: torch.Size([1126, 48, 22]) torch.Size([1126])
Testing Shape: torch.Size([142, 48, 22]) torch.Size([142])


In [27]:
class LSTM(nn.Module):
    
    def __init__(self, num_classes, input_size, hidden_size, num_layers):
        super().__init__()
        self.num_classes = num_classes # output size
        self.num_layers = num_layers # number of recurrent layers in the lstm
        self.input_size = input_size # input size
        self.hidden_size = hidden_size # neurons in each lstm layer
        # LSTM model
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True, dropout=0.2) # lstm
        self.fc_1 =  nn.Linear(hidden_size, 128) # fully connected 
        self.fc_2 = nn.Linear(128, num_classes) # fully connected last layer
        self.relu = nn.ReLU()
        
    def forward(self,x):
        # hidden state
        h_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # cell state
        c_0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size))
        # propagate input through LSTM
        output, (hn, cn) = self.lstm(x, (h_0, c_0)) # (input, hidden, and internal state)
        hn = hn.view(-1, self.hidden_size) # reshaping the data for Dense layer next
        out = self.relu(hn)
        out = self.fc_1(out) # first dense
        out = self.relu(out) # relu
        out = self.fc_2(out) # final output
        return out

In [28]:
def training_loop(n_epochs, lstm, optimiser, loss_fn, X_train, y_train,
                  X_test, y_test):
    for epoch in range(n_epochs):
        lstm.train()
        outputs = lstm.forward(X_train) # forward pass
        optimiser.zero_grad() # calculate the gradient, manually setting to 0
        # obtain the loss function
        loss = loss_fn(outputs, y_train)
        loss.backward() # calculates the loss of the loss function
        optimiser.step() # improve from loss, i.e backprop
        # test loss
        lstm.eval()
        test_preds = lstm(X_test)
        test_loss = loss_fn(test_preds, y_test)
        if epoch % 100 == 0:
            print("Epoch: %d, train loss: %1.5f, test loss: %1.5f" % (epoch, 
                                                                      loss.item(), 
                                                                      test_loss.item())) 


In [29]:
import warnings
warnings.filterwarnings('ignore')

n_epochs = 1000 # 1000 epochs
learning_rate = 0.001 # 0.001 lr

input_size = 22 # number of features
hidden_size = 2 # number of features in hidden state
num_layers = 1 # number of stacked lstm layers

num_classes = 1 # number of output classes 

lstm = LSTM(num_classes, 
              input_size, 
              hidden_size, 
              num_layers)

loss_fn = torch.nn.MSELoss()    # mean-squared error for regression
optimiser = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

training_loop(n_epochs=n_epochs,
              lstm=lstm,
              optimiser=optimiser,
              loss_fn=loss_fn,
              X_train=X_train_tensors_final,
              y_train=Y_train_tensors,
              X_test=X_test_tensors_final,
              y_test=Y_test_tensors)

Epoch: 0, train loss: nan, test loss: nan
Epoch: 100, train loss: nan, test loss: nan
Epoch: 200, train loss: nan, test loss: nan
Epoch: 300, train loss: nan, test loss: nan
Epoch: 400, train loss: nan, test loss: nan
Epoch: 500, train loss: nan, test loss: nan
Epoch: 600, train loss: nan, test loss: nan
Epoch: 700, train loss: nan, test loss: nan
Epoch: 800, train loss: nan, test loss: nan
Epoch: 900, train loss: nan, test loss: nan
